In [2]:
# Dependencies
%matplotlib inline

import matplotlib.pyplot as plt
import json
import requests
import os
import pandas as pd
import numpy as np
import gmaps
import requests

from scipy.stats import linregress
from dotenv import load_dotenv
from citipy import citipy
from datetime import datetime
import time
load_dotenv()
api_key = os.getenv("OpenWeather_API_KEY")
g_key = os.getenv("g_key")

# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
# Create a list for holding the cities.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key
url

'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7818e16cca2b62a3109d0a324d8b034e'

In [4]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

782

In [19]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
       
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

city_url

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | aguas vermelhas
Processing Record 2 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 3 of Set 1 | adrar
Processing Record 4 of Set 1 | albany
Processing Record 5 of Set 1 | mataura
Processing Record 6 of Set 1 | tiksi
Processing Record 7 of Set 1 | kupang
Processing Record 8 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 9 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 10 of Set 1 | osypenko
Processing Record 11 of Set 1 | pisco
Processing Record 12 of Set 1 | bonavista
Processing Record 13 of Set 1 | hobart
Processing Record 14 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 15 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 16 of Set 1 | puerto ayora
Processing Record 17 of Set 1 | tecpan
Processing Record 18 of Set 1 | ca mau
Processing Record 19 of Set 1 | ushuaia
Processing Record 20 of Set 1

Processing Record 40 of Set 4 | pilar
Processing Record 41 of Set 4 | tieling
Processing Record 42 of Set 4 | kamsack
Processing Record 43 of Set 4 | lavrentiya
Processing Record 44 of Set 4 | sao joao da barra
Processing Record 45 of Set 4 | kerema
Processing Record 46 of Set 4 | namibe
Processing Record 47 of Set 4 | airai
Processing Record 48 of Set 4 | svay rieng
Processing Record 49 of Set 4 | jiddah
City not found. Skipping...
Processing Record 50 of Set 4 | agirish
Processing Record 1 of Set 5 | grand gaube
Processing Record 2 of Set 5 | brazzaville
Processing Record 3 of Set 5 | sao filipe
Processing Record 4 of Set 5 | chimore
Processing Record 5 of Set 5 | abashiri
Processing Record 6 of Set 5 | anadyr
Processing Record 7 of Set 5 | bolungarvik
City not found. Skipping...
Processing Record 8 of Set 5 | westport
Processing Record 9 of Set 5 | spencer
Processing Record 10 of Set 5 | oxilithos
Processing Record 11 of Set 5 | lensk
Processing Record 12 of Set 5 | manono
Processin

Processing Record 28 of Set 8 | nhulunbuy
Processing Record 29 of Set 8 | burica
City not found. Skipping...
Processing Record 30 of Set 8 | flinders
Processing Record 31 of Set 8 | dalvik
Processing Record 32 of Set 8 | parati
Processing Record 33 of Set 8 | mao
Processing Record 34 of Set 8 | conde
Processing Record 35 of Set 8 | newport
Processing Record 36 of Set 8 | pedasi
Processing Record 37 of Set 8 | sigli
Processing Record 38 of Set 8 | charters towers
Processing Record 39 of Set 8 | palmer
Processing Record 40 of Set 8 | yantzaza
Processing Record 41 of Set 8 | manokwari
Processing Record 42 of Set 8 | shumskiy
Processing Record 43 of Set 8 | comodoro rivadavia
Processing Record 44 of Set 8 | vardo
Processing Record 45 of Set 8 | san jeronimo
Processing Record 46 of Set 8 | black river
Processing Record 47 of Set 8 | suhbaatar
Processing Record 48 of Set 8 | alzamay
Processing Record 49 of Set 8 | hasaki
Processing Record 50 of Set 8 | gondanglegi
Processing Record 1 of Set 

Processing Record 17 of Set 12 | gerede
Processing Record 18 of Set 12 | tagusao
Processing Record 19 of Set 12 | seoul
Processing Record 20 of Set 12 | san lorenzo
Processing Record 21 of Set 12 | hualmay
Processing Record 22 of Set 12 | severo-kurilsk
Processing Record 23 of Set 12 | nanakuli
Processing Record 24 of Set 12 | civitavecchia
Processing Record 25 of Set 12 | bloomingdale
Processing Record 26 of Set 12 | lodwar
Processing Record 27 of Set 12 | kon tum
Processing Record 28 of Set 12 | namwala
Processing Record 29 of Set 12 | tres arroyos
Processing Record 30 of Set 12 | saraland
Processing Record 31 of Set 12 | thai nguyen
Processing Record 32 of Set 12 | xuddur
Processing Record 33 of Set 12 | kuvshinovo
Processing Record 34 of Set 12 | kalmunai
Processing Record 35 of Set 12 | namikupa
Processing Record 36 of Set 12 | lufilufi
Processing Record 37 of Set 12 | sattahip
Processing Record 38 of Set 12 | polewali
Processing Record 39 of Set 12 | stromness
Processing Record 4

City not found. Skipping...
Processing Record 5 of Set 16 | paita
Processing Record 6 of Set 16 | rawannawi
City not found. Skipping...
Processing Record 7 of Set 16 | abha
Processing Record 8 of Set 16 | carutapera
Processing Record 9 of Set 16 | khorixas
Processing Record 10 of Set 16 | sambava
Processing Record 11 of Set 16 | tawagan
Processing Record 12 of Set 16 | rizhao
Processing Record 13 of Set 16 | maloy
Processing Record 14 of Set 16 | letnyaya stavka
Processing Record 15 of Set 16 | inongo
Processing Record 16 of Set 16 | ballina
Processing Record 17 of Set 16 | upington
Processing Record 18 of Set 16 | lagunillas
Processing Record 19 of Set 16 | tsiroanomandidy
Processing Record 20 of Set 16 | wagar
Processing Record 21 of Set 16 | polyarnyy
Processing Record 22 of Set 16 | hay river
Processing Record 23 of Set 16 | zainsk
-----------------------------
Data Retrieval Complete      
-----------------------------


'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7818e16cca2b62a3109d0a324d8b034e&q=zainsk'

In [5]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Description"]

city_data_df

NameError: name 'city_data' is not defined

In [22]:
# Create the output file (CSV).
output_data_file = "weather_database/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")